In [1]:
import csv
import random
from utils import *
from collections import defaultdict, Counter
import os
import time
import tqdm
import matplotlib.pyplot as plt
#import seaborn as sns
import numpy as np

## generate SBMs and Erdos-Renyi graph

In [2]:
class SBMGraphStream():
    '''
    The class of Graph Stream from the stochastic block model
    ----- Parameters -----
    # n_vertex: the number of vertices in the graph
    # p_intra: the probability for + edge (u,v) for the same cluster
    # p_inter: the probability for + edge (u,v) for different clusters
    # k_cluster: number of clusters in the clustering
    ----- Methods ----
    # read_next_edge(): read the next edge and move the index +1
    ----- Representation ----
    The graph is representation with an indexed array of vertices and a dictionary with (u_i, u_j): labels
    '''
    
    def __init__(self, n_vertex, p_intra=0.8, p_inter=0.2, k_cluster=7):
        '''
        :param n_vertex: the the number of vertices in the graph
        '''
        self.n_vertex = n_vertex
        self.p_intra = p_intra
        self.p_inter = p_inter
        self.k_cluster = k_cluster
        
        # initialize the vertex set and the cluster labels
        self.vertex_set = np.array([self.n_vertex])
        num_v_per_cluster = n_vertex//self.k_cluster
        n_residual = n_vertex % num_v_per_cluster
        cluster_labels_list = []
        for i_cluster in range(k_cluster):
            cluster_labels_list.append(i_cluster*np.ones([num_v_per_cluster]))
        if n_residual!=0:
            cluster_labels_list.append((k_cluster-1)*np.ones([n_residual]))
        # collect them as a 1-d array
        self.cluster_labels = np.reshape(np.hstack(cluster_labels_list).astype(int), [-1])
        # initialize the edges -- using +1 and -1 to represent the edge labels
        # also compute the cost
        self.cc_cost = 0
        self.edge_dict = {}
        for u_i in tqdm.tqdm(range(self.n_vertex)):
            for u_j in np.arange(u_i+1, self.n_vertex):
                if self.cluster_labels[u_i] == self.cluster_labels[u_j]:
                    if np.random.rand() <= p_intra:
                        self.edge_dict[(u_i,u_j)] = 1
                    else:
                        self.edge_dict[(u_i,u_j)] = -1
                        self.cc_cost = self.cc_cost + 1
                else:
                    if np.random.rand() <= p_inter:
                        self.edge_dict[(u_i,u_j)] = 1
                        self.cc_cost = self.cc_cost + 1
                    else:
                        self.edge_dict[(u_i,u_j)] = -1
        # randomize the order of edge arrival
        self.edge_names = list(self.edge_dict.keys())
        random.shuffle(self.edge_names)
        self.num_edges = len(self.edge_names)
        # maintain a pointer of the number of edges
        self.current_stream_ind = 0
        
    def read_next_edge(self):
        
        this_edge_name = self.edge_names[self.current_stream_ind]
        this_edge_label = self.edge_dict[this_edge_name]
        self.current_stream_ind = self.current_stream_ind + 1
        if self.current_stream_ind>=self.num_edges-1:
            return None, None
        
        return this_edge_name, this_edge_label
    
    def write_edges(self, write_path=None):
        if not write_path:
            raise ValueError('the writing path has to be specified!')
        file_name = 'SBM_n='+str(self.n_vertex)+'_p='+str(self.p_intra)+'_k=' + str(self.k_cluster) +'.csv'
        with open(os.path.join(write_path, file_name), 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            for edge in self.edge_dict:
                if self.edge_dict[edge]>0:
                    writer.writerow([f'{edge[0]} {edge[1]}'])
    
    def reset_index(self):
        '''
        reset the pointer
        '''
        self.current_stream_ind = 0

In [185]:
sbm_graph_stream = SBMGraphStream(n_vertex=200,p_intra=0.95, p_inter=0.05, k_cluster=4)

100%|██████████| 200/200 [00:00<00:00, 4468.56it/s]


In [186]:
sbm_graph_stream.write_edges(write_path='../data/')

## The functions that implements our algorithm

In [187]:
def test_sparse_vertex(current_graph, eps=0.2, induce_vertex_list=None):
    # sample log n neighbors for each vertex
    current_num_vertex = len(current_graph)
    num_sample = max((int)(np.log(current_num_vertex)/eps), 20)
    num_diff_test_sample = max((int)(np.log(current_num_vertex)/eps), 20)
    sample_dict = {}
    for this_vertex in current_graph:
        sample_vertex_set = list(current_graph[this_vertex].getRandom(i=num_sample))
        sample_dict[this_vertex] = sample_vertex_set
    sparse_vertex_list = []
    # test sparsity for each vertex
    for this_vertex in current_graph:
        if current_graph[this_vertex].degree == 0:
            parse_vertex_list.append(this_vertex)
            continue
        num_neighbor_diff = 0
        neighbors_this = sample_dict[this_vertex]
        for comp_vertex in sample_dict[this_vertex]:
            neighbors_comp = sample_dict[comp_vertex]
            sample_vertex = np.random.choice(current_num_vertex, num_diff_test_sample)
            # the intersections of the neighbor
            total_diff = 0
            for test_vertex in sample_vertex:
                if (test_vertex in current_graph[this_vertex]) and (test_vertex not in current_graph[comp_vertex]):
                    total_diff = total_diff + 1
                if (test_vertex in current_graph[comp_vertex]) and (test_vertex not in current_graph[this_vertex]):
                    total_diff = total_diff + 1
            if (total_diff>=eps*num_diff_test_sample):
                num_neighbor_diff = num_neighbor_diff + 1
        if num_neighbor_diff >=eps*num_sample:
#             print('==========================')
#             print(num_neighbor_diff)
#             print(num_sample)
            sparse_vertex_list.append(this_vertex)
    
    # return the list of sparse vertices
    return sparse_vertex_list

In [196]:
def sparse_dense_decop(current_graph, eps=0.2, induce_vertex_list=None):
    current_num_vertex = len(current_graph)
    # the returned clusters
    SDD_clustering = {}
    # check sparse vertices
    tic = time.time()
    current_sparse_vertice = test_sparse_vertex(current_graph, eps=eps)
    for sparse_vertex in current_sparse_vertice:
        SDD_clustering[sparse_vertex]=sparse_vertex
    # print('The time for sparsity testing is ', time.time()-tic)
    # sample from the dense vertices
    dense_subgraph = {vertex: current_graph[vertex] 
                      for vertex in current_graph if vertex not in current_sparse_vertice}
    anchor_vertex_dict = {}
    # tic = time.time()
    for this_vertex in dense_subgraph:
        # rejection sampling
        dense_sample_prob = max((np.log(current_num_vertex))/(eps*dense_subgraph[this_vertex].degree), 0.02)
        if dense_sample_prob>=np.random.uniform(low=0.0, high=1.0):
            anchor_vertex_dict[this_vertex] = dense_subgraph[this_vertex]
    # print('The time for indexing the subgraph ', time.time()-tic)
    # recursively form almost-cliques
    num_sample = max((int)(5*np.log(current_num_vertex)), 20)
    AC_dict = {'AC':{}, 'size': 0, 'counter': 0}
    # maintain a list of covered vertices
    # tic = time.time()
    covered_AC_vertex = []
    for this_anchor_vertex in anchor_vertex_dict.keys():
        if this_anchor_vertex in covered_AC_vertex:
            continue
        AC_dict[this_anchor_vertex] = []
        SDD_clustering[this_anchor_vertex] = this_anchor_vertex  # assign to the cluster represented by self
        covered_AC_vertex.append(this_anchor_vertex)
        anchor_neighbor_samples = list(anchor_vertex_dict[this_anchor_vertex].getRandom(i=num_sample))
        for candidate_vertex in dense_subgraph[this_anchor_vertex]:
            if (candidate_vertex in current_sparse_vertice) or (candidate_vertex in covered_AC_vertex):
                continue
            # test whether their symmetric difference is large enough
            # the intersections of the neighbor
            total_diff = 0
            for anchor_neighbor in anchor_neighbor_samples:
                if anchor_neighbor not in list(dense_subgraph[candidate_vertex]):
                    total_diff = total_diff + 1
#             anchor_neighbor_in_sample = np.intersect1d(sample_vertex, list(dense_subgraph[this_anchor_vertex]))
#             cand_neighbor_in_sample = np.intersect1d(sample_vertex, list(dense_subgraph[candidate_vertex]))
#             dif1 = np.setdiff1d(anchor_neighbor_in_sample, cand_neighbor_in_sample)
#             dif2 = np.setdiff1d(cand_neighbor_in_sample, anchor_neighbor_in_sample)
#             if sbm_graph_stream.cluster_labels[this_anchor_vertex]!=sbm_graph_stream.cluster_labels[candidate_vertex]:
#                 print('============= Different cluster happens! ==================')
#                 print(anchor_neighbor_samples)
#                 print(cand_neighbor_samples)
#                 print(len(dif1[0]))
#                 print(len(dif2[0]))
#             total_diff = len(dif1)+len(dif2)
            if (total_diff<=1.5*eps*num_sample):
                AC_dict[this_anchor_vertex].append(candidate_vertex)
                SDD_clustering[candidate_vertex] = this_anchor_vertex # assign the candidate vertex to the anchor
                covered_AC_vertex.append(candidate_vertex)
            # this line is for debugging purpose -- remove later
            else:
                pass
    # add codes to add vertices to the almost-cliques
    # print('The time for forming almost-cliques is ', time.time()-tic)
    # merge undecided vertices to the almost-cliques
    num_diff_test_sample = max((int)(2*np.log(current_num_vertex)), 20)
    # pull the undecided vertices
    undecided_AC_vertices = []
    for dense_vertex in dense_subgraph.keys():
        if dense_vertex not in SDD_clustering.keys():
            undecided_AC_vertices.append(dense_vertex)
    for und_vertex in undecided_AC_vertices:
        und_v_sampled_neighbors = list(dense_subgraph[und_vertex].getRandom(i=num_diff_test_sample))
        neighbor_AC_names = []
        for merge_test_vertex in und_v_sampled_neighbors:
            if merge_test_vertex not in SDD_clustering:
                continue
            neighbor_AC_names.append(SDD_clustering[merge_test_vertex])
        AC_counts = Counter(neighbor_AC_names)
        most_common_AC, most_common_AC_freq = AC_counts.most_common(1)[0]
        if most_common_AC_freq>=(1-2*eps)*num_diff_test_sample:
            SDD_clustering[und_vertex] = most_common_AC
            AC_dict[most_common_AC].append(und_vertex)
        else:
            # this line should not happen, but just add in case of strange bugs
            current_sparse_vertice.append(und_vertex)
            SDD_clustering[und_vertex] = und_vertex
    
    
    return current_sparse_vertice, AC_dict, SDD_clustering, anchor_vertex_dict

In [197]:
def singleton_cluster_alg(current_graph):
    return {vertex: vertex for vertex in current_graph}

## The implementation of dynamic algorithms

In [198]:
def extract_induced_subgraph(current_graph, target_vertex):
    num_vertex = len(current_graph)
    neighbor_of_target = list(current_graph[target_vertex])
    num_sample = min(max((int)(2*np.log(current_num_vertex)), 20), len(neighbor_of_target))
    induced_subgraph = {}
    for neighbor_vertex in neighbor_of_target:
        # pretend that intersection happens in O(1) time
        this_neighbor_list = list(set(current_graph[neighbor_vertex].a).intersection(set(neighbor_of_target)))
        if len(this_neighbor_list)>=0.7*current_graph[neighbor_vertex].degree:
            induced_subgraph[neighbor_vertex] = OptList(a=this_neighbor_list,
                                                        d=this_neighbor_list, 
                                                        degree=current_graph[neighbor_vertex].degree)
        # estimation of the cost
#         sample_vertex_in_subgraph = random.sample(neighbor_of_target, num_sample)
#         count_degree = 0
#         for sampled_vertex in sample_vertex_in_subgraph:
#             if sampled_vertex in current_graph[neighbor_vertex]:
#                 count_degree = count_degree + 1
#         est_degree = count_degree*len(neighbor_of_target)/num_sample
#         induced_subgraph[neighbor_vertex] = OptList(a=this_neighbor_list, d=this_neighbor_list, degree=est_degree)
    
    return induced_subgraph

In [ ]:
def dynamic_sparse_dense_decomp(subgraph, global_graph, global_SDD, target_vertex):
    '''
    params: 
    subgraph: the subgraph that we want to run SDD on. Note that this is *not* the exact N(u) since we
                filter out all the vertices that are not dense locally
    global_graph: the global graph represented by the dictionary of OptList()
    global_SDD: 
    '''
    # first step: run SDD on the subgraph
    subgraph_sparse_vertex_list,subgraph_AC_dict, subgraph_SDD_clustering, _ = sparse_dense_decop(subgraph)
    # add new vertices to the almost-cliques if they in sparse vertices
    

## Read the edges and maintain clustering

In [199]:
graph_file_name="../data/SBM_n=200_p=0.95_k=4.csv"
# graph_file_name= "../data/email-Eu-core.csv"

In [200]:
adjacency_list, edge_list = create_graph_from_csv(graph_file_name)

In [201]:
'''
TODO

See if functions are doable

'''

'\nTODO\n\nSee if functions are doable\n\n'

In [202]:
no_edges = len(edge_list)  # No. of edges

prob_del = 0.2      # Probability to delete edge
eps_param = 0.4

current_graph = {}
current_edge_list = []

available_edge_list = np.random.permutation(edge_list).tolist()

stream_length = (int)(0.5*no_edges)

track_update_num = {}
track_update_benckmark = {}

max_ite = 10000

SDD_amortized_update_time = 0
pivot_amortized_update_time = 0

# initialize the SDD clustering with singletons
SDD_clustering = singleton_cluster_alg(adjacency_list)

for i in range(stream_length):
    # Insertion
    if available_edge_list: #
        current_edge_list.append(available_edge_list[i])
        u = available_edge_list[i][0]
        v = available_edge_list[i][1]
        if u not in current_graph.keys():
            current_graph[u] = OptList()
        current_graph[u].insert(v)
        if v not in current_graph.keys():
            current_graph[v] = OptList()
        current_graph[v].insert(u)
        available_edge_list.pop(0)
        
        # keep track of the benchmark for the updates
        if u not in track_update_benckmark:
            track_update_benckmark[u] = current_graph[u].degree
        if v not in track_update_benckmark:
            track_update_benckmark[v] = current_graph[v].degree
        # update the tracking of the updates on u and v
        if u not in track_update_num:
            track_update_num[u] = 1
        else:
            track_update_num[u] = track_update_num[u] + 1
            
        if v not in track_update_num:
            track_update_num[v] = 1
        else:
            track_update_num[v] = track_update_num[v] + 1
        
         
        '''
        Code for SDD and PIVOT goes here
        '''
        # SDD clustering
        # Maintain the original clustering -- if the newly added vertice are not in the SDD
        # then we assign singletons
        if u not in SDD_clustering:
            SDD_clustering[u] = u
        if v not in SDD_clustering:
            SDD_clustering[v] = v
        # The update step
        if (track_update_num[u]>max(2, eps_param*track_update_benckmark[u])):
            # tests
            start_SDD = time.time()
            current_sparse_vertex_list, almost_cliques, SDD_clustering, anchor_vertex_dict = sparse_dense_decop(current_graph, eps=eps_param)
            SDD_amortized_update_time = (time.time() - start_SDD)/track_update_num[u]
            track_update_num[u] = 0
            track_update_benckmark[u] = current_graph[u].degree
        start_pivot = time.time()
        pivot_clustering = classical_pivot(current_graph)
        end_pivot = time.time()
        pivot_amortized_update_time = end_pivot - start_pivot
        singleton_clustering = singleton_cluster_alg(adjacency_list)
        # clear the number of updates
        # track_update_num[u] = 0
        # track_update_benckmark = current_graph[u].degree
        # =========== TODO: add this as a test for whether the SDD succeeds ======== 
#         all_vertex_list = [v for v in current_graph]
#         AC_vertex_list = []
#         for anchor_ver in almost_cliques.keys():
#             AC_vertex_list.append(anchor_ver)
#             for ac_ver in almost_cliques[anchor_ver]:
#                 AC_vertex_list.append(ac_ver)
#         AC_vertex_list = list(set(AC_vertex_list))
#         recovered_vertex = np.concatenate((AC_vertex_list, current_sparse_vertex_list))
#         print('===============================')
#         print(np.setdiff1d(all_vertex_list,recovered_vertex))
#         print(len(current_sparse_vertex_list))
#         print(len(anchor_vertex_dict.keys()))
#         print('******************************')
#         print('The number of almost-cliques is ', len(almost_cliques))
        if i>500:
            SDD_cost = correlation_clustering_value(current_graph, SDD_clustering)
#             all_vertex_list = [v for v in current_graph]
#             AC_vertex_list = []
#             for anchor_ver in almost_cliques.keys():
#                 AC_vertex_list.append(anchor_ver)
#                 for ac_ver in almost_cliques[anchor_ver]:
#                     AC_vertex_list.append(ac_ver)
#             AC_vertex_list = list(set(AC_vertex_list))
#             recovered_vertex = np.concatenate((AC_vertex_list, current_sparse_vertex_list))
#             print('===============================')
#             print(np.setdiff1d(all_vertex_list,recovered_vertex))
#             print(len(current_sparse_vertex_list))
#             print(len(anchor_vertex_dict.keys()))
#             print('******************************')
#             print('The number of almost-cliques is ', len(almost_cliques))
            pivot_cost = correlation_clustering_value(current_graph, pivot_clustering)
            singleton_cost = correlation_clustering_value(current_graph, singleton_clustering)
            print('SDD clustering cost is', SDD_cost, 'and the running time is', SDD_amortized_update_time)
            print('Pivot clustering cost is', pivot_cost, 'and the running time is', pivot_amortized_update_time)
            print('Singleton clustering cost is', singleton_cost)
#         if "SBM" in graph_file_name:
#             print('The correct optimal clustering cost should be', sbm_graph_stream.cc_cost)
        if i>max_ite:
            all_vertex_list = [v for v in current_graph]
            AC_vertex_list = []
            for anchor_ver in almost_cliques.keys():
                AC_vertex_list.append(anchor_ver)
                for ac_ver in almost_cliques[anchor_ver]:
                    AC_vertex_list.append(ac_ver)
            AC_vertex_list = list(set(AC_vertex_list))
            recovered_vertex = np.concatenate((AC_vertex_list, current_sparse_vertex_list))
            print('===============================')
            print(np.setdiff1d(all_vertex_list,recovered_vertex))
            print(len(current_sparse_vertex_list))
            print(len(anchor_vertex_dict.keys()))
            print('******************************')
            print('The number of almost-cliques is ', len(almost_cliques))
            break
        
#     else:
#         # We have run out of edges to insert
#         edge_to_delete = np.random.choice(current_edge_list)
        
#         u = edge_to_delete[0]
#         v = edge_to_delete[1]
#         current_graph[u].remove(v)
#         current_graph[v].remove(u)
        
#         available_edge_list.extend(edge_to_delete)
#         current_edge_list.remove(edge_to_delete)
    
        
    
    
#     if np.random.binomial(1,prob_del):
#         # Deletion
#         print(current_edge_list)
#         edge_to_delete = np.random.choice(current_edge_list)
        
#         u = edge_to_delete[0]
#         v = edge_to_delete[1]
#         current_graph[u].remove(v)
#         current_graph[v].remove(u)
        
#         available_edge_list.extend(edge_to_delete)
#         current_edge_list.remove(edge_to_delete)

SDD clustering cost is 629.5 and the running time is 0.005545997619628906
Pivot clustering cost is 601.5 and the running time is 0.0002589225769042969
Singleton clustering cost is 492.0
SDD clustering cost is 630.5 and the running time is 0.005545997619628906
Pivot clustering cost is 649.0 and the running time is 0.00026297569274902344
Singleton clustering cost is 493.0
SDD clustering cost is 631.5 and the running time is 0.005545997619628906
Pivot clustering cost is 636.5 and the running time is 0.0002799034118652344
Singleton clustering cost is 494.0
SDD clustering cost is 632.5 and the running time is 0.005545997619628906
Pivot clustering cost is 592.5 and the running time is 0.0002830028533935547
Singleton clustering cost is 495.0
SDD clustering cost is 633.5 and the running time is 0.005545997619628906
Pivot clustering cost is 625.0 and the running time is 0.0003070831298828125
Singleton clustering cost is 496.0
SDD clustering cost is 634.5 and the running time is 0.00554599761962

SDD clustering cost is 702.0 and the running time is 0.00792849063873291
Pivot clustering cost is 666.5 and the running time is 0.0002429485321044922
Singleton clustering cost is 545.0
SDD clustering cost is 703.0 and the running time is 0.00792849063873291
Pivot clustering cost is 681.5 and the running time is 0.0002601146697998047
Singleton clustering cost is 546.0
SDD clustering cost is 704.0 and the running time is 0.00792849063873291
Pivot clustering cost is 690.5 and the running time is 0.0002541542053222656
Singleton clustering cost is 547.0
SDD clustering cost is 705.0 and the running time is 0.010936657587687174
Pivot clustering cost is 692.0 and the running time is 0.0002570152282714844
Singleton clustering cost is 548.0
SDD clustering cost is 706.0 and the running time is 0.0049944718678792315
Pivot clustering cost is 688.0 and the running time is 0.00025010108947753906
Singleton clustering cost is 549.0
SDD clustering cost is 707.0 and the running time is 0.0049944718678792

SDD clustering cost is 756.5 and the running time is 0.006756019592285156
Pivot clustering cost is 741.0 and the running time is 0.00026988983154296875
Singleton clustering cost is 585.0
SDD clustering cost is 757.5 and the running time is 0.008577048778533936
Pivot clustering cost is 749.0 and the running time is 0.0002849102020263672
Singleton clustering cost is 586.0
SDD clustering cost is 756.5 and the running time is 0.011210282643636068
Pivot clustering cost is 763.0 and the running time is 0.0003037452697753906
Singleton clustering cost is 587.0
SDD clustering cost is 757.5 and the running time is 0.011210282643636068
Pivot clustering cost is 722.0 and the running time is 0.0002651214599609375
Singleton clustering cost is 588.0
SDD clustering cost is 758.5 and the running time is 0.005634188652038574
Pivot clustering cost is 728.0 and the running time is 0.00027632713317871094
Singleton clustering cost is 589.0
SDD clustering cost is 759.5 and the running time is 0.0113834540049

SDD clustering cost is 796.0 and the running time is 0.005015577588762555
Pivot clustering cost is 791.0 and the running time is 0.00026297569274902344
Singleton clustering cost is 629.0
SDD clustering cost is 797.0 and the running time is 0.005015577588762555
Pivot clustering cost is 756.5 and the running time is 0.00026798248291015625
Singleton clustering cost is 630.0
SDD clustering cost is 798.0 and the running time is 0.012704690297444662
Pivot clustering cost is 802.5 and the running time is 0.0002770423889160156
Singleton clustering cost is 631.0
SDD clustering cost is 799.0 and the running time is 0.012674093246459961
Pivot clustering cost is 840.5 and the running time is 0.0002491474151611328
Singleton clustering cost is 632.0
SDD clustering cost is 800.0 and the running time is 0.012674093246459961
Pivot clustering cost is 767.5 and the running time is 0.0002532005310058594
Singleton clustering cost is 633.0
SDD clustering cost is 801.0 and the running time is 0.0093230009078

SDD clustering cost is 872.5 and the running time is 0.004213439093695747
Pivot clustering cost is 869.5 and the running time is 0.0002529621124267578
Singleton clustering cost is 678.0
SDD clustering cost is 873.5 and the running time is 0.004213439093695747
Pivot clustering cost is 862.0 and the running time is 0.0002827644348144531
Singleton clustering cost is 679.0
SDD clustering cost is 874.5 and the running time is 0.004969507455825806
Pivot clustering cost is 811.0 and the running time is 0.0002570152282714844
Singleton clustering cost is 680.0
SDD clustering cost is 875.5 and the running time is 0.004969507455825806
Pivot clustering cost is 849.5 and the running time is 0.00026702880859375
Singleton clustering cost is 681.0
SDD clustering cost is 876.5 and the running time is 0.01241302490234375
Pivot clustering cost is 874.0 and the running time is 0.0002570152282714844
Singleton clustering cost is 682.0
SDD clustering cost is 877.5 and the running time is 0.01241302490234375


...

SDD clustering cost is 2706.0 and the running time is 0.00832781195640564
Pivot clustering cost is 3444.0 and the running time is 0.00030493736267089844
Singleton clustering cost is 3932.0
SDD clustering cost is 2706.0 and the running time is 0.00832781195640564
Pivot clustering cost is 3573.0 and the running time is 0.0002739429473876953
Singleton clustering cost is 3932.0
SDD clustering cost is 2705.0 and the running time is 0.00832781195640564
Pivot clustering cost is 3501.0 and the running time is 0.0002741813659667969
Singleton clustering cost is 3933.0
SDD clustering cost is 2705.0 and the running time is 0.00832781195640564
Pivot clustering cost is 3510.0 and the running time is 0.000286102294921875
Singleton clustering cost is 3933.0
SDD clustering cost is 2748.0 and the running time is 0.010122946330479212
Pivot clustering cost is 3270.0 and the running time is 0.00026798248291015625
Singleton clustering cost is 3933.0
SDD clustering cost is 2748.0 and the running time is 0.01

SDD clustering cost is 2688.5 and the running time is 0.01006630750802847
Pivot clustering cost is 3475.0 and the running time is 0.0002808570861816406
Singleton clustering cost is 3958.0
SDD clustering cost is 2687.5 and the running time is 0.01006630750802847
Pivot clustering cost is 3525.0 and the running time is 0.0002999305725097656
Singleton clustering cost is 3959.0
SDD clustering cost is 2687.5 and the running time is 0.01006630750802847
Pivot clustering cost is 3495.0 and the running time is 0.0002942085266113281
Singleton clustering cost is 3959.0
SDD clustering cost is 2688.5 and the running time is 0.01006630750802847
Pivot clustering cost is 3387.0 and the running time is 0.0002930164337158203
Singleton clustering cost is 3960.0
SDD clustering cost is 2687.5 and the running time is 0.01006630750802847
Pivot clustering cost is 3378.0 and the running time is 0.0003020763397216797
Singleton clustering cost is 3961.0
SDD clustering cost is 2688.5 and the running time is 0.0100

SDD clustering cost is 2591.0 and the running time is 0.00873507559299469
Pivot clustering cost is 3499.5 and the running time is 0.00026702880859375
Singleton clustering cost is 3981.0
SDD clustering cost is 2590.0 and the running time is 0.00873507559299469
Pivot clustering cost is 3097.5 and the running time is 0.00030803680419921875
Singleton clustering cost is 3982.0
SDD clustering cost is 2555.0 and the running time is 0.008762852350870768
Pivot clustering cost is 3434.5 and the running time is 0.0002930164337158203
Singleton clustering cost is 3982.0
SDD clustering cost is 2555.0 and the running time is 0.008762852350870768
Pivot clustering cost is 3458.5 and the running time is 0.0002918243408203125
Singleton clustering cost is 3982.0
SDD clustering cost is 2554.0 and the running time is 0.008762852350870768
Pivot clustering cost is 3449.5 and the running time is 0.0003070831298828125
Singleton clustering cost is 3983.0
SDD clustering cost is 2553.0 and the running time is 0.00

SDD clustering cost is 2932.0 and the running time is 0.008297471439137179
Pivot clustering cost is 3337.5 and the running time is 0.0005540847778320312
Singleton clustering cost is 4010.0
SDD clustering cost is 2931.0 and the running time is 0.008297471439137179
Pivot clustering cost is 3376.0 and the running time is 0.0006811618804931641
Singleton clustering cost is 4011.0
SDD clustering cost is 2930.0 and the running time is 0.008297471439137179
Pivot clustering cost is 3150.0 and the running time is 0.0003409385681152344
Singleton clustering cost is 4012.0
SDD clustering cost is 2930.0 and the running time is 0.008297471439137179
Pivot clustering cost is 3566.0 and the running time is 0.000324249267578125
Singleton clustering cost is 4012.0
SDD clustering cost is 2930.0 and the running time is 0.008297471439137179
Pivot clustering cost is 3192.5 and the running time is 0.0004570484161376953
Singleton clustering cost is 4012.0
SDD clustering cost is 2930.0 and the running time is 0.

SDD clustering cost is 2746.0 and the running time is 0.009872862270900182
Pivot clustering cost is 3473.0 and the running time is 0.0002930164337158203
Singleton clustering cost is 4030.0
SDD clustering cost is 2685.0 and the running time is 0.011103391647338867
Pivot clustering cost is 3176.5 and the running time is 0.00031113624572753906
Singleton clustering cost is 4030.0
SDD clustering cost is 2684.0 and the running time is 0.011103391647338867
Pivot clustering cost is 2995.5 and the running time is 0.00027489662170410156
Singleton clustering cost is 4031.0
SDD clustering cost is 2684.0 and the running time is 0.011103391647338867
Pivot clustering cost is 3424.5 and the running time is 0.0002880096435546875
Singleton clustering cost is 4031.0
SDD clustering cost is 2657.0 and the running time is 0.010718143903292142
Pivot clustering cost is 3353.5 and the running time is 0.0003311634063720703
Singleton clustering cost is 4032.0
SDD clustering cost is 2657.0 and the running time is

For of clustering is dict[vertex-name]: cluster-name